In [1]:
from typing import List, Dict

from oop_functions.experiment_helper import *
from oop_functions.experiment_runner import ExperimentRunner
from oop_functions.util_functions import *
from oop_functions.visualization_util import *
from oop_functions.analytics_cv_util import *

%matplotlib inline
import warnings

warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd

from xgboost import XGBClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

import itertools

from dcurves import dca, plot_graphs


In [2]:
processed_data = pd.read_csv('./processed_dataset/recent_propagated_dataset.csv', index_col=0)
screen_data = pd.read_csv('./dataset/Ovarian/Screening/ovar_screen_data_mar22_d032222.csv')
screen_abnorm_data = pd.read_csv('./dataset/Ovarian/Screening Abnormalities/ovar_scrsub_data_mar22_d032222.csv')

In [3]:
unique_ids = screen_data['plco_id'].unique()

In [4]:
len(unique_ids)


30849

In [36]:
len(screen_data)

150993

In [71]:
len(screen_abnorm_data)

20186

In [37]:

processed_data = convert_numeric_to_float16(processed_data)

In [38]:
experiment_data_helpers_inst: Dict[str, ExperimentDataHelper] = {}

In [39]:
experiment_data_helpers: List[ExperimentDataHelper] = [
    # ExperimentDataHelperSingleLabelScreenedFirst5,    
    # ExperimentDataHelperScreenedFirst5,
    ExperimentDataHelperNotScreenedColsFirst5,
    # ExperimentDataHelperScreenedColsFirst5,
    # ExperimentDataHelperScreenedOrCancer,
    # ExperimentDataHelperScreened,
    # ExperimentDataHelperNotScreenedCols,
    # ExperimentDataHelperScreenedCols,
    # ExperimentDataHelperAll,
    # ExperimentDataHelperSingleLabelScreenedOrCancer,
    # ExperimentDataHelperSingleLabelScreened,
    # ExperimentDataHelperSingleLabelNotScreenedCols,
    # ExperimentDataHelperSingleLabelScreenedCols,
    # ExperimentDataHelperSingleLabelAll,
    # ExperimentDataHelperScreenedFirst5ca125AndBinary,
    # ExperimentDataHelperScreenedFirst5ca125Binary, 
    # ExperimentDataHelperScreenedFirst5ca125AndBinaryNoResult,
]

In [40]:
years = [1]
num_folds=15
train_size = 25000
classifier_name = "STUB_CLASSIFIER"
for year in years:
    for experiment_data_helper_class in experiment_data_helpers:
        if year:
            label = f'cancer_in_next_{year}_years'
        else: 
            label = 'ovar_cancer'
        filesuffix = f'_for_experiment_{experiment_data_helper_class.get_name()}_{classifier_name}_{label}__{num_folds}_trials'
        select_cols = None
        print(filesuffix)
        # print(select_cols)
        experiment_data_helper: ExperimentDataHelper = experiment_data_helper_class(processed_data, label, ['cancer_'], train_size = train_size, select_features=select_cols)
        experiment_data_helpers_inst[filesuffix] = experiment_data_helper
            

_for_experiment_not_screened_cols_first_5_STUB_CLASSIFIER_cancer_in_next_1_years__15_trials


In [66]:
source_df = processed_data

In [67]:
source_df = source_df[['plco_id', 'was_screened']]
source_df = source_df.drop_duplicates()

In [68]:
print_df(get_column_values_count(source_df, 'was_screened'))

+----+----------------+---------+
|    |   was_screened |   count |
|----+----------------+---------|
|  0 |              0 |   47360 |
|  1 |              1 |   30849 |
+----+----------------+---------+


In [44]:
30849 / (30849 + 47360)

0.3944430947844877

In [89]:
source_df = experiment_data_helper.source_df

In [46]:
print_df(get_column_values_count(source_df, 'is_ca125_screening_record'))

+----+-----------------------------+---------+
|    |   is_ca125_screening_record |   count |
|----+-----------------------------+---------|
|  0 |                           0 |  113452 |
|  1 |                           1 |  150988 |
+----+-----------------------------+---------+


In [69]:
150988 / (150988 + 113452)

0.5709726213885947

In [90]:
source_df = source_df[source_df['is_ca125_screening_record'] == 1]

In [49]:
# 150758
# 150998
# 240

In [87]:
112 / 150998

0.000741731678565279

In [51]:
print_df(get_column_values_count(source_df, 'ovar_cancer'))
print_df(get_column_values_count(source_df, 'cancer_in_next_1_years'))

+----+---------------+---------+
|    |   ovar_cancer |   count |
|----+---------------+---------|
|  0 |             0 |  149792 |
|  1 |             1 |    1196 |
+----+---------------+---------+
+----+--------------------------+---------+
|    |   cancer_in_next_1_years |   count |
|----+--------------------------+---------|
|  0 |                        0 |  150876 |
|  1 |                        1 |     112 |
+----+--------------------------+---------+


In [95]:
source_df = convert_numeric_to_float16(source_df)
source_df['age'] 

In [96]:
summary = summarize_features(source_df)

In [97]:
print_df(summary)

+---------------------------+----------------+---------+---------------+--------------+----------+------------+-------------+-------------+-------------+
|                           |   unique count |   count |          mean |          std |      min |        25% |         50% |         75% |         max |
|---------------------------+----------------+---------+---------------+--------------+----------+------------+-------------+-------------+-------------|
| ovar_cancer               |              2 |  150988 |      0        |     0        |   0      |     0      |      0      |      0      |      1      |
| ovar_behavior             |              3 |    1196 |      2.99414  |     0.129028 |   1      |     3      |      3      |      3      |      3      |
| ovar_histtype             |             11 |    1196 |     10.3594   |    13.0234   |   1      |     2      |      2      |     31      |     39      |
| plco_id                   |          30849 |     nan |    nan        |   n

In [112]:
ages = source_df['age'].dropna().values.astype(float)

In [113]:
np.min(ages)

50.0

In [114]:
np.mean(ages)

62.526988899780115

In [115]:
np.std(ages)

5.628949086774989

In [105]:
source_df[['age']].fillna(60).describe()

,age
count,150988.0
mean,NaN
std,0.0
min,50.0
25%,60.0
50%,60.0
75%,65.0
max,80.0


In [106]:
np.mean(source_df['age'].dropna())

nan

In [82]:
source_df = experiment_data_helper.source_df

In [83]:
source_df = source_df[source_df['is_ca125_screening_record'] == 1]
source_df = source_df[source_df['ovar_observe_year'] <= 5]

In [84]:
source_df = source_df.drop(['age', 'study_yr', 'ovar_observe_year', 'is_ca125_screening_record', 'is_ultra_screening_record', 'index'], axis=1)
# source_df = source_df[]
source_df = source_df.drop_duplicates()

In [85]:
print_df(get_column_values_count(source_df, 'race7'))
print_df(get_column_values_count(source_df, 'ovar_cancer'))
print_df(get_column_values_count(source_df, 'cancer_in_next_1_years'))

+----+---------+---------+
|    |   race7 |   count |
|----+---------+---------|
|  0 |       1 |   27222 |
|  1 |       2 |    1666 |
|  2 |       3 |     458 |
|  3 |       4 |    1077 |
|  4 |       5 |     138 |
|  5 |       6 |      88 |
|  6 |       7 |     283 |
+----+---------+---------+
+----+---------------+---------+
|    |   ovar_cancer |   count |
|----+---------------+---------|
|  0 |             0 |   30557 |
|  1 |             1 |     375 |
+----+---------------+---------+
+----+--------------------------+---------+
|    |   cancer_in_next_1_years |   count |
|----+--------------------------+---------|
|  0 |                        0 |   30820 |
|  1 |                        1 |     112 |
+----+--------------------------+---------+


In [86]:
122 / (122 + 30820)

0.003942860836403594

In [57]:
source_df = experiment_data_helper.source_df

In [58]:
len(source_df[(source_df['ovar_observe_year'] <= 5) & (source_df['is_ca125_screening_record'] == 0) & (source_df['is_ultra_screening_record'] == 0) & (source_df['was_screened'] == 1)])

18702

In [59]:
len(source_df[(source_df['ovar_observe_year'] <= 5) & ((source_df['is_ca125_screening_record'] == 1) | (source_df['is_ultra_screening_record'] == 1))])

150988

In [60]:
len(source_df[((source_df['is_ca125_screening_record'] == 1) | (source_df['is_ultra_screening_record'] == 1))])

150988

In [61]:
len(source_df[((source_df['is_ca125_screening_record'] == 1))])

150988

In [62]:
len(source_df[(source_df['ovar_observe_year'] <= 5) & (source_df['was_screened'] == 1)])

169690

In [63]:
len(source_df[(source_df['ovar_observe_year'] <= 5) & ((source_df['is_ca125_screening_record'] == 1) | (source_df['is_ultra_screening_record'] == 1))])

150988